In [1]:
import pandas as pd

anime_df = pd.read_csv('Data/anime/anime.csv')

print(anime_df.head())
print(anime_df.describe())

   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  
           anime_id        rating       members
count  12294.000000  12064.000000  1.229400e+04
mean   14058.221653      6.473902  1.807134e+04
st

In [2]:
anime_df = anime_df.dropna()

# Convert the 'genre' column to a list of genres
anime_df['genre'] = anime_df['genre'].apply(lambda x: x.split(', '))

# One-hot encode the genres
anime_df = anime_df.join(anime_df.pop('genre').str.join('|').str.get_dummies())

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(anime_df.drop(['name', 'anime_id', 'type', 'episodes', 'rating', 'members'], axis=1))

# Create a lookup table to get anime names from their indices and vice versa
indices = pd.Series(anime_df.index, index=anime_df['name']).drop_duplicates()

In [4]:
def recommend_anime(name, cosine_sim=cosine_sim):
    # Get the index of the anime
    idx = indices[name]

    # Get the pairwsie similarity scores of all animes with that anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the animes based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar animes
    sim_scores = sim_scores[1:11]

    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar animes
    return anime_df['name'].iloc[anime_indices]


In [5]:
print(recommend_anime('Naruto'))

615                                    Naruto: Shippuuden
841                                                Naruto
1103    Boruto: Naruto the Movie - Naruto ga Hokage ni...
1343                                          Naruto x UT
1472          Naruto: Shippuuden Movie 4 - The Lost Tower
1573    Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...
2458                 Naruto Shippuuden: Sunny Side Battle
2997    Naruto Soyokazeden Movie: Naruto to Mashin to ...
175                                Katekyo Hitman Reborn!
7628                              Kyutai Panic Adventure!
Name: name, dtype: object
